In [1]:
import os
import warnings
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image


In [3]:
# Suppress warnings
warnings.filterwarnings("ignore")

In [23]:
# Define the paths to the dataset
train_dir = r'C:\Users\User\.jupyter\plant disease\plant disease\train'  # Update this to your actual path
val_dir = r'C:\Users\User\.jupyter\plant disease\plant disease\valid'      # Update this to your actual path

In [25]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True
)

In [27]:
# For validation, just rescale
val_datagen = ImageDataGenerator(rescale=1./255)

In [1]:
# Create generators for training and validation datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


NameError: name 'train_datagen' is not defined

In [31]:
# Check the number of classes
num_classes = len(train_generator.class_indices)
print("Number of classes:", num_classes)


Number of classes: 5


In [33]:
# Define the model architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Output layer matches number of classes
])


In [35]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [37]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  # Adjust the number of epochs as needed
    steps_per_epoch=train_generator.samples // 32,
    validation_steps=val_generator.samples // 32
)


Epoch 1/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 385s 1s/step - accuracy: 0.4342 - loss: 1.2665 - val_accuracy: 0.7812 - val_loss: 0.6245
Epoch 2/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 1s 514us/step - accuracy: 0.7812 - loss: 0.4903 - val_accuracy: 0.8000 - val_loss: 0.4658
Epoch 3/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 226s 778ms/step - accuracy: 0.8090 - loss: 0.4992 - val_accuracy: 0.7943 - val_loss: 0.5957
Epoch 4/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 1s 337us/step - accuracy: 0.9375 - loss: 0.1875 - val_accuracy: 0.6667 - val_loss: 0.9151
Epoch 5/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 329s 1s/step - accuracy: 0.8896 - loss: 0.3170 - val_accuracy: 0.9149 - val_loss: 0.2196
Epoch 6/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 1s 318us/step - accuracy: 0.9375 - loss: 0.1568 - val_accuracy: 1.0000 - val_loss: 0.0664
Epoch 7/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 228s 784ms/step - accuracy: 0.8999 - loss: 0.2712 - val_accuracy: 0.7595 - val_loss: 0.8652
Epoch 8/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 1s 322us/step - accuracy: 0.9062 - loss: 0.2707 

In [59]:
# Save the model (optional)
model.save('tomato_disease_classifier.h5')


In [53]:
# Updated dictionary for disease to pesticide recommendations
pesticide_recommendations = {
    'Tomato___Bacterial_spot': 'Use Copper-based fungicides',
    'Tomato___Early_blight': 'Use Chlorothalonil or Azoxystrobin',
    'Tomato___healthy': 'No pesticides needed',
    'Tomato___Late_blight': 'Use Mancozeb or Ridomil',
    'Tomato___Leaf_Mold': 'Use Potassium bicarbonate or Neem oil'
}

def recommend_pesticide(disease):
    return pesticide_recommendations.get(disease, "No recommendation available.")


In [63]:
def predict_and_recommend(image_path):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # Make predictions
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)
    
    # Map predicted class to disease name
    disease_name = list(train_generator.class_indices.keys())[predicted_class[0]]
    
    # Get pesticide recommendation
    pesticide = recommend_pesticide(disease_name)
    
    return disease_name, pesticide

# Example usage
image_path = r'C:\Users\User\.jupyter\plant disease\plant disease\train\Tomato___Bacterial_spot\0e94696b-3e0d-4d4c-a712-01197e228cf1___UF.GRC_BS_Lab Leaf 8641.JPG'  # Update this to your actual test image path
disease, pesticide = predict_and_recommend(image_path)
print(f'Detected Disease: {disease}\nRecommended Pesticide: {pesticide}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Detected Disease: Tomato___Bacterial_spot
Recommended Pesticide: Use Copper-based fungicides
